In [876]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
import os
import torch

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [4]:
import pickle
with open('../intermediate/jgsvect.pickle','rb') as file:
    jgsvect = pickle.load(file)
with open('../intermediate/y.pickle','rb') as file:
    y = pickle.load(file)
with open('../intermediate/x.pickle','rb') as file:
    x = pickle.load(file)

In [5]:
jgsvect = [txt for jgs in jgsvect for txt in jgs]
labels = [lst for jgs in y for lst in jgs]

In [14]:
asp = {
"NAME" : 1,
"CITATION" : 2,
"COUNSEL" : 3,
"JUDGE" : 4,
"FACTS" : 5,
"RLC" : 6,
"REASONING" : 7,
"ARG" : 8,
"STATUTE" : 9,
"PRECEDENT" : 10,
"RPC" : 11,
"ISSUE" : 12
}

In [6]:
dict = {0 : 0, 1 : 0, 2 : 0, 3 : 0, 4 : 0, 5 : 0, 6 : 0, 7 : 0, 8 : 0, 9 : 0, 10 : 0, 11 : 0, 12 : 0}
for i in range(0,len(labels)):
    for label in labels[i]:
        dict[label]+=1

In [16]:
for key in asp:
    print(f"Aspect {key} : {dict[asp[key]]} sentences")

Aspect NAME : 12 sentences
Aspect CITATION : 13 sentences
Aspect COUNSEL : 28 sentences
Aspect JUDGE : 15 sentences
Aspect FACTS : 401 sentences
Aspect RLC : 68 sentences
Aspect REASONING : 665 sentences
Aspect ARG : 44 sentences
Aspect STATUTE : 87 sentences
Aspect PRECEDENT : 104 sentences
Aspect RPC : 98 sentences
Aspect ISSUE : 15 sentences


In [12]:
dict[5]

401

In [824]:
mlb.fit(labels)
labels = mlb.fit_transform(labels)

In [857]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.layers import SpatialDropout1D, Dropout
 

model = Sequential()
#model.add(Embedding(10000,300,input_length=768))
model.add(LSTM(units=100, input_shape=(768,1), return_sequences=True))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(units=13, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [846]:
temp = []
for vect in jgsvect:
    temp.append(vect.numpy())
X_final = temp.copy()

X_final = np.array(X_final)

In [832]:
X_final[0].shape

(768,)

In [833]:
Y_final = labels.copy()

In [834]:
print(type(X_final))
print(type(Y_final))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [835]:
print(X_final.shape, Y_final.shape)

(1741, 768) (1741, 13)


In [836]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final,Y_final,test_size=0.20,random_state=42)

In [838]:
print(X_test.shape)
print(X_train.shape)

(349, 768)
(1392, 768)


In [874]:
dict[7]

0

In [858]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=32)

Epoch 1/10
44/44 [==============================] - 19s 407ms/step - loss: 2.2339 - accuracy: 0.3111 - val_loss: 1.9930 - val_accuracy: 0.3524
Epoch 2/10
44/44 [==============================] - 17s 397ms/step - loss: 1.9985 - accuracy: 0.3362 - val_loss: 2.0039 - val_accuracy: 0.3524
Epoch 3/10
44/44 [==============================] - 17s 398ms/step - loss: 2.0095 - accuracy: 0.3226 - val_loss: 2.0025 - val_accuracy: 0.3524
Epoch 4/10
44/44 [==============================] - 17s 396ms/step - loss: 1.9971 - accuracy: 0.3384 - val_loss: 1.9938 - val_accuracy: 0.3524
Epoch 5/10
44/44 [==============================] - 17s 396ms/step - loss: 1.9891 - accuracy: 0.3333 - val_loss: 2.0028 - val_accuracy: 0.3524
Epoch 6/10
44/44 [==============================] - 17s 397ms/step - loss: 1.9979 - accuracy: 0.3276 - val_loss: 1.9886 - val_accuracy: 0.3524
Epoch 7/10
44/44 [==============================] - 17s 396ms/step - loss: 2.0039 - accuracy: 0.3254 - val_loss: 1.9903 - val_accuracy: 0.3524

In [859]:
# 3. Make predictions
predictions = model.predict(X_test)

11/11 [==============================] - 2s 113ms/step


In [862]:
predictions[10]

array([0.1547631 , 0.00410143, 0.00577291, 0.01556682, 0.00667626,
       0.2369559 , 0.04002809, 0.36337447, 0.02119171, 0.04380893,
       0.05048272, 0.049382  , 0.00789566], dtype=float32)

In [863]:
predictions[0]

array([0.15346925, 0.00405928, 0.00569847, 0.01543377, 0.0066705 ,
       0.23810655, 0.03986674, 0.3644772 , 0.02115727, 0.04370851,
       0.05008907, 0.04940014, 0.00786333], dtype=float32)

In [864]:
predictions[1]

array([0.15208836, 0.00400935, 0.00562291, 0.01527784, 0.0066566 ,
       0.23921125, 0.03968285, 0.36576614, 0.02110804, 0.04361974,
       0.04972133, 0.04940495, 0.00783062], dtype=float32)

In [865]:
print(f"Model trained on {len(X_train)} sentences")
print(f"Total number of sentences in test data : {len(X_test)}")
for i in range(1,7):
    x = [np. argsort(temp)[::-1][:i] for temp in predictions]
    indices_list = []

    # Iterate through each binary vector
    for binary_vector in Y_test:
        # Use a list comprehension to get the indices of 1s
        indices = [i for i, value in enumerate(binary_vector) if value == 1]
        indices_list.append(indices)
    
    ispresent = []
    for j in range(len(x)):
        present = False
        for label in indices_list[j]:
            if label in x[j]:
                present = True
                break
        if present:
            ispresent.append(1)
        else:
            ispresent.append(0)
    print(f"Number of sentences whose labels are present in top {i} predicted aspects for that sentence: {np.sum(ispresent)} | accuracy : {np.sum(ispresent)*100/len(ispresent)}%")


Model trained on 1392 sentences
Total number of sentences in test data : 349
Number of sentences whose labels are present in top 1 predicted aspects for that sentence: 140 | accuracy : 40.11461318051576%
Number of sentences whose labels are present in top 2 predicted aspects for that sentence: 197 | accuracy : 56.44699140401146%
Number of sentences whose labels are present in top 3 predicted aspects for that sentence: 263 | accuracy : 75.35816618911174%
Number of sentences whose labels are present in top 4 predicted aspects for that sentence: 286 | accuracy : 81.9484240687679%
Number of sentences whose labels are present in top 5 predicted aspects for that sentence: 302 | accuracy : 86.53295128939828%
Number of sentences whose labels are present in top 6 predicted aspects for that sentence: 314 | accuracy : 89.97134670487107%


In [647]:
x = [np. argsort(temp)[::-1][:3] for temp in predictions]

In [648]:
x[4]

array([7, 5, 0])

In [649]:
Y_test[4]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [650]:
x[1]

array([7, 5, 0])

## PREDICT

In [792]:
with open('../intermediate/test.pickle','rb') as file:
    temp = pickle.load(file)

In [795]:
predictions = model.predict((temp.numpy()))

4/4 [==============================] - 0s 54ms/step


In [17]:
predictions

NameError: name 'predictions' is not defined

In [598]:
predicted_labels = [np. argsort(dum)[::-1][:1] for dum in predictions]

In [607]:
predicted_labels

[array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),
 array([7]),

In [601]:
asp = {
"NAME" : 1,
"CITATION" : 2,
"COUNSEL" : 3,
"JUDGE" : 4,
"FACTS" : 5,
"RLC" : 6,
"REASONING" : 7,
"ARG" : 8,
"STATUTE" : 9,
"PRECEDENT" : 10,
"RPC" : 11,
"ISSUE" : 12
}

In [602]:
aspect = asp["FACTS"]

In [606]:
aspect

5

In [604]:
with open('../intermediate/jgs.pickle','rb') as file:
    jgs = pickle.load(file)

In [605]:
for i in range(len(predict_judgements)):
    if aspect in predicted_labels[i]:
        print(jgs[i])